<p> Regression experiments on all dataset, where Environment is filtered </p>

In [1]:
import keras
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_percentage_error, max_error, median_absolute_error, mean_absolute_error
from sklearn.metrics import mean_squared_error
import time


2023-11-20 17:02:05.262852: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-20 17:02:05.296331: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-20 17:02:05.296368: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-20 17:02:05.297117: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-20 17:02:05.302343: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-20 17:02:05.302807: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [2]:
df = pd.read_csv('data/dataMoney.csv')

In [3]:
df.shape

(2020, 15)

In [4]:
df.head()

,max_discharge_sum_r1,max_discharge_sum_r2_1,max_discharge_sum_r2_2,max_discharge_sum_r3,flood_sum_r1,flood_sum_r2_1,flood_sum_r2_2,flood_sum_r3,HW_inund_uncertainty,bridge_scour_uncertainty,travel_demand_uncertainty,restoration_capacity_uncertainty,dir_costs,lost_trips_costs_s_r,traffic_costs_s_r
0,443.457856,1658.236820,1658.442362,924.207396,7704.283023,30959.150563,19494.507734,18060.945333,1.237179,1.066024,1.178643,7,2.396527e+07,3.771578e+07,1.346267e+07
1,525.178784,1658.659798,1658.659798,376.518772,8042.074336,30948.775455,19495.723976,12105.892278,0.805171,0.920064,1.189389,5,2.156163e+07,3.629843e+07,1.427793e+07
2,525.178784,1658.659798,1658.659798,376.518772,8042.074336,30948.775455,19495.723976,12105.892278,0.895325,1.052411,0.782405,5,2.187945e+07,2.450664e+07,-6.527196e+06
3,528.794113,1659.966169,1660.085687,376.518762,8056.793807,30980.988568,19504.576908,12106.087505,0.748587,0.955364,1.143878,6,2.195877e+07,3.199478e+07,1.273019e+07
4,523.393056,1660.295347,1660.307923,379.856164,8055.296723,30984.114890,19505.358354,12140.315910,0.965187,0.831900,1.344699,6,2.236592e+07,3.822993e+07,2.754356e+07


In [157]:
dummy_columns = pd.get_dummies(df['Environment'], prefix='class')
dummy_columns

,class_Benign,class_Low,class_Severe,class_moderate
0,True,False,False,False
1,True,False,False,False
2,True,False,False,False
3,True,False,False,False
4,True,False,False,False
...,...,...,...,...
113095,False,False,True,False
113096,False,False,True,False
113097,False,False,True,False
113098,False,False,True,False


In [158]:
df = pd.concat([df, dummy_columns], axis=1)
df.head()

,Unnamed: 0,Day,SC,Age,Environment,Span_number,h_NoDamage,h_Minor,h_Moderate,h_Extensive,h_Severe,Res_Minor,Res_moderate,Res_extensive,Res_Severe,Resilience,class_Benign,class_Low,class_Severe,class_moderate
0,0,1,0.0,20,Benign,1,0.96478,0.002324,0.003064,0.003209,0.026623,0.90173,0.7126,0.39687,0.11078,0.97344,True,False,False,False
1,1,2,0.0,20,Benign,1,0.96478,0.002324,0.003064,0.003209,0.026623,0.90173,0.7126,0.39687,0.11078,0.97340,True,False,False,False
2,2,3,0.0,20,Benign,1,0.96478,0.002324,0.003064,0.003209,0.026623,0.90173,0.7126,0.39687,0.11078,0.97337,True,False,False,False
3,3,4,0.0,20,Benign,1,0.96478,0.002324,0.003064,0.003209,0.026623,0.90173,0.7126,0.39687,0.11078,0.97334,True,False,False,False
4,4,5,0.0,20,Benign,1,0.96478,0.002324,0.003064,0.003209,0.026623,0.90173,0.7126,0.39687,0.11078,0.97332,True,False,False,False


In [159]:
df = df.drop("Environment", axis=1)

In [160]:
df.head()

,Unnamed: 0,Day,SC,Age,Span_number,h_NoDamage,h_Minor,h_Moderate,h_Extensive,h_Severe,Res_Minor,Res_moderate,Res_extensive,Res_Severe,Resilience,class_Benign,class_Low,class_Severe,class_moderate
0,0,1,0.0,20,1,0.96478,0.002324,0.003064,0.003209,0.026623,0.90173,0.7126,0.39687,0.11078,0.97344,True,False,False,False
1,1,2,0.0,20,1,0.96478,0.002324,0.003064,0.003209,0.026623,0.90173,0.7126,0.39687,0.11078,0.97340,True,False,False,False
2,2,3,0.0,20,1,0.96478,0.002324,0.003064,0.003209,0.026623,0.90173,0.7126,0.39687,0.11078,0.97337,True,False,False,False
3,3,4,0.0,20,1,0.96478,0.002324,0.003064,0.003209,0.026623,0.90173,0.7126,0.39687,0.11078,0.97334,True,False,False,False
4,4,5,0.0,20,1,0.96478,0.002324,0.003064,0.003209,0.026623,0.90173,0.7126,0.39687,0.11078,0.97332,True,False,False,False


In [161]:
X = df.drop('Resilience', axis=1)
y = df['Resilience']

In [162]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [163]:
from sklearn.preprocessing import MaxAbsScaler

scaler = MaxAbsScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [164]:
from sklearn.metrics import mean_absolute_error
from numpy import sqrt

def print_metrics(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    print(f'Mean Squared Error (MSE):              {mse:.10f}')
    
    rmse = sqrt(mse)
    print(f'Root Mean Squared Error (RMSE):        {rmse:.10f}')
    
    mae = mean_absolute_error(y_true, y_pred)
    print(f'Mean Absolute Error (MAE):             {mae:.10f}')
    
    r2 = r2_score(y_true, y_pred)
    print(f'R-squared (R²):                        {r2:.10f}')
    
    mape = mean_absolute_percentage_error(y_true, y_pred)
    print(f'Mean Absolute Percentage Error (MAPE): {mape:.10f}')    
    
    me = max_error(y_true, y_pred)
    print(f'Max Error (ME):                        {me:.10f}')    
    
    med_ae = median_absolute_error(y_true, y_pred)
    print(f'Median Absolute Error (MedAE):         {med_ae:.10f}')    

<h2> Random Forest </h2>

In [165]:
from sklearn.ensemble import RandomForestRegressor

rf_model = RandomForestRegressor(n_estimators=5, random_state=0)
start_time = time.time()
rf_model.fit(X_train, y_train)
print(f'Training time: {time.time() - start_time:.4f} seconds')

Training time: 1.9523 seconds


In [166]:
print("Random Forest: train")
y_pred = rf_model.predict(X_train)
print_metrics(y_train, y_pred)
print()
print("Random Forest: test")
y_pred = rf_model.predict(X_test)
print_metrics(y_test, y_pred)


Random Forest: train
Mean Squared Error (MSE):              0.0000001644
Root Mean Squared Error (RMSE):        0.0004054561
Mean Absolute Error (MAE):             0.0001081598
R-squared (R²):                        0.9999964703
Mean Absolute Percentage Error (MAPE): 0.0003215164
Max Error (ME):                        0.0134300000
Median Absolute Error (MedAE):         0.0000000000

Random Forest: test
Mean Squared Error (MSE):              0.0000006667
Root Mean Squared Error (RMSE):        0.0008165427
Mean Absolute Error (MAE):             0.0002391688
R-squared (R²):                        0.9999859015
Mean Absolute Percentage Error (MAPE): 0.0007077370
Max Error (ME):                        0.0164000000
Median Absolute Error (MedAE):         0.0000120000


<h2> Support vector regression </h2>

In [167]:
from sklearn.svm import SVR
svr_model = SVR(kernel='rbf', gamma='auto', coef0=0.0, epsilon=0.001, max_iter=-1)
start_time = time.time()
svr_model.fit(X_train, y_train)
print(f'Training time: {time.time() - start_time:.4f} seconds')

Training time: 515.4814 seconds


In [168]:
print("SVR: train")
y_pred = svr_model.predict(X_train)
print_metrics(y_train, y_pred)
print()
print("SVR: test")
y_pred = svr_model.predict(X_test)
print_metrics(y_test, y_pred)

SVR: train
Mean Squared Error (MSE):              0.0000030513
Root Mean Squared Error (RMSE):        0.0017467999
Mean Absolute Error (MAE):             0.0007848737
R-squared (R²):                        0.9999344854
Mean Absolute Percentage Error (MAPE): 0.0035837985
Max Error (ME):                        0.0208464315
Median Absolute Error (MedAE):         0.0005822975

SVR: test
Mean Squared Error (MSE):              0.0000031979
Root Mean Squared Error (RMSE):        0.0017882743
Mean Absolute Error (MAE):             0.0007917225
R-squared (R²):                        0.9999323785
Mean Absolute Percentage Error (MAPE): 0.0035885071
Max Error (ME):                        0.0202485132
Median Absolute Error (MedAE):         0.0005790361


<h2> Ridge regression </h2>

In [169]:
from sklearn.linear_model import Ridge, Lasso

ridge_model = Ridge(alpha=1.0)  # alpha - regularization strength
start_time = time.time()
ridge_model.fit(X_train, y_train)
print(f'Training time: {time.time() - start_time:.4f} seconds')


Training time: 0.0104 seconds


In [170]:
print("Ridge regression: train")
y_pred = ridge_model.predict(X_train)
print_metrics(y_train, y_pred)
print()
print("Ridge regression: test")
y_pred = ridge_model.predict(X_test)
print_metrics(y_test, y_pred)

Ridge regression: train
Mean Squared Error (MSE):              0.0004481467
Root Mean Squared Error (RMSE):        0.0211694758
Mean Absolute Error (MAE):             0.0100566151
R-squared (R²):                        0.9903778526
Mean Absolute Percentage Error (MAPE): 0.0331054998
Max Error (ME):                        0.2865205758
Median Absolute Error (MedAE):         0.0054686971

Ridge regression: test
Mean Squared Error (MSE):              0.0004642888
Root Mean Squared Error (RMSE):        0.0215473622
Mean Absolute Error (MAE):             0.0101664057
R-squared (R²):                        0.9901824141
Mean Absolute Percentage Error (MAPE): 0.0331930709
Max Error (ME):                        0.2783343989
Median Absolute Error (MedAE):         0.0054746698


<h2> Lasso regression </h2>

In [171]:
# Ridge Regression
lasso_model = Lasso(alpha=1.0)  # You can adjust the regularization strength (alpha) as needed
start_time = time.time()
lasso_model.fit(X_train, y_train)
print(f'Training time: {time.time() - start_time:.4f} seconds')

Training time: 0.0228 seconds


In [172]:
print("Lasso regression: train")
y_pred = lasso_model.predict(X_train)
print_metrics(y_train, y_pred)
print()
print("Lasso regression: test")
y_pred = lasso_model.predict(X_test)
print_metrics(y_test, y_pred)

Lasso regression: train
Mean Squared Error (MSE):              0.0465745003
Root Mean Squared Error (RMSE):        0.2158112609
Mean Absolute Error (MAE):             0.1727905913
R-squared (R²):                        0.0000000000
Mean Absolute Percentage Error (MAPE): 0.8063239324
Max Error (ME):                        0.7069192772
Median Absolute Error (MedAE):         0.1529420178

Lasso regression: test
Mean Squared Error (MSE):              0.0472922164
Root Mean Squared Error (RMSE):        0.2174677364
Mean Absolute Error (MAE):             0.1738668237
R-squared (R²):                        -0.0000141682
Mean Absolute Percentage Error (MAPE): 0.8074512839
Max Error (ME):                        0.7058285852
Median Absolute Error (MedAE):         0.1527520178


<h2> GRNN </h2>

In [173]:
import math
from sklearn.base import BaseEstimator, ClassifierMixin


class GRNN(BaseEstimator, ClassifierMixin):
    def __init__(self, name = "GRNN", sigma = 0.1):
        self.name = name
        self.sigma = 2 * np.power(sigma, 2)
        
    def predict(self, instance_X, train_X, train_y):
        gausian_distances = np.exp(-np.power(np.sqrt((np.square(train_X-instance_X).sum(axis=1))),2)\
                                   / self.sigma)
        gausian_distances_sum = gausian_distances.sum()
        if gausian_distances_sum < math.pow(10, -7): gausian_distances_sum = math.pow(10, -7)
        result = np.multiply(gausian_distances, train_y).sum() / gausian_distances_sum
        return result

In [174]:
for s in np.arange(0.1, 0.5, 0.05):
    grnn = GRNN(sigma=s)
    start_time = time.time()
    predictions = np.apply_along_axis(lambda i: grnn.predict(i, X_train, y_train), axis=1, arr=X_test)
    print("--- %s seconds ---" % (time.time() - start_time))
    print("Sigma: " + str(s))
    print_metrics(y_test, predictions)


--- 148.09039282798767 seconds ---
Sigma: 0.1
Mean Squared Error (MSE):              0.0000441910
Root Mean Squared Error (RMSE):        0.0066476275
Mean Absolute Error (MAE):             0.0033268987
R-squared (R²):                        0.9990655634
Mean Absolute Percentage Error (MAPE): 0.0137217325
Max Error (ME):                        0.0691675092
Median Absolute Error (MedAE):         0.0017165052
--- 145.92745304107666 seconds ---
Sigma: 0.15000000000000002
Mean Squared Error (MSE):              0.0001184921
Root Mean Squared Error (RMSE):        0.0108854072
Mean Absolute Error (MAE):             0.0065991450
R-squared (R²):                        0.9974944340
Mean Absolute Percentage Error (MAPE): 0.0248563229
Max Error (ME):                        0.0880888281
Median Absolute Error (MedAE):         0.0033085863
--- 142.58426213264465 seconds ---
Sigma: 0.20000000000000004
Mean Squared Error (MSE):              0.0003374758
Root Mean Squared Error (RMSE):        0.018370513

<h3> Stochastic Gradient Descent Regressor </h3>

In [175]:
from sklearn.linear_model import SGDRegressor
sgd_model = SGDRegressor(loss = 'squared_epsilon_insensitive', alpha=0.0001)

start_time = time.time()
sgd_model.fit(X_train, y_train)
print(f'Training time: {time.time() - start_time:.4f} seconds')

Training time: 0.0873 seconds


In [176]:
print("SGD Regressor: train")
y_pred = sgd_model.predict(X_train)
print_metrics(y_train, y_pred)
print()
print("SGD Regressor: test")
y_pred = sgd_model.predict(X_test)
print_metrics(y_test, y_pred)

SGD Regressor: train
Mean Squared Error (MSE):              0.0034610971
Root Mean Squared Error (RMSE):        0.0588310892
Mean Absolute Error (MAE):             0.0505815931
R-squared (R²):                        0.9256868665
Mean Absolute Percentage Error (MAPE): 0.2743351882
Max Error (ME):                        0.1679455793
Median Absolute Error (MedAE):         0.0485520781

SGD Regressor: test
Mean Squared Error (MSE):              0.0035156530
Root Mean Squared Error (RMSE):        0.0592929425
Mean Absolute Error (MAE):             0.0510327721
R-squared (R²):                        0.9256600111
Mean Absolute Percentage Error (MAPE): 0.2757879899
Max Error (ME):                        0.1610374807
Median Absolute Error (MedAE):         0.0492618718


<h2> ADA Boost regressor </h2>

In [177]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor

adaboost_model = AdaBoostRegressor(DecisionTreeRegressor(max_depth=4), n_estimators=300)

start_time = time.time()
adaboost_model.fit(X_train, y_train)
print(f'Training time: {time.time() - start_time:.4f} seconds')

Training time: 48.6902 seconds


In [178]:
print("ADA boost Regressor: train")
y_pred = adaboost_model.predict(X_train)
print_metrics(y_train, y_pred)
print()
print("ADA boost Regressor: test")
y_pred = adaboost_model.predict(X_test)
print_metrics(y_test, y_pred)

ADA boost Regressor: train
Mean Squared Error (MSE):              0.0008641493
Root Mean Squared Error (RMSE):        0.0293964161
Mean Absolute Error (MAE):             0.0260216374
R-squared (R²):                        0.9814458712
Mean Absolute Percentage Error (MAPE): 0.1698761424
Max Error (ME):                        0.0471583071
Median Absolute Error (MedAE):         0.0261933874

ADA boost Regressor: test
Mean Squared Error (MSE):              0.0008638308
Root Mean Squared Error (RMSE):        0.0293909988
Mean Absolute Error (MAE):             0.0260340934
R-squared (R²):                        0.9817339276
Mean Absolute Percentage Error (MAPE): 0.1698212696
Max Error (ME):                        0.0471583071
Median Absolute Error (MedAE):         0.0261777066


<h2> Multi-layer Perceptron regressor </h2>

In [179]:
from sklearn.neural_network import MLPRegressor
mlp_model = MLPRegressor(hidden_layer_sizes=(100, 40, 20), activation='relu', solver='adam', alpha=0.0001, batch_size='auto', max_iter=200)

start_time = time.time()
mlp_model.fit(X_train, y_train)
print(f'Training time: {time.time() - start_time:.4f} seconds')

Training time: 6.1206 seconds


In [180]:
print("MLP Regressor: train")
y_pred = mlp_model.predict(X_train)
print_metrics(y_train, y_pred)
print()
print("MLP Regressor: test")
y_pred = mlp_model.predict(X_test)
print_metrics(y_test, y_pred)

MLP Regressor: train
Mean Squared Error (MSE):              0.0000154568
Root Mean Squared Error (RMSE):        0.0039315161
Mean Absolute Error (MAE):             0.0029502529
R-squared (R²):                        0.9996681270
Mean Absolute Percentage Error (MAPE): 0.0132747500
Max Error (ME):                        0.0337289005
Median Absolute Error (MedAE):         0.0023646941

MLP Regressor: test
Mean Squared Error (MSE):              0.0000156744
Root Mean Squared Error (RMSE):        0.0039590857
Mean Absolute Error (MAE):             0.0029524289
R-squared (R²):                        0.9996685590
Mean Absolute Percentage Error (MAPE): 0.0132798880
Max Error (ME):                        0.0295596231
Median Absolute Error (MedAE):         0.0023501250
